In [71]:
%matplotlib widget

In [72]:
import matplotlib.pyplot as plt

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import re

In [4]:
meat_balance_sheet = pd.read_csv('World-Livestock/FAOSTAT-Meat-Food-Balance-Sheets.csv', encoding = "ISO-8859-1")

In [5]:
livestock = pd.read_csv('World-Livestock/FAOSTAT-Production-Livestock.csv', encoding = "ISO-8859-1")

In [6]:
population = pd.read_csv('World-Livestock/WORLDBANK-World-Population.csv', encoding = "ISO-8859-1")

First lets examine the livestock data...

In [7]:
livestock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170682 entries, 0 to 170681
Data columns (total 11 columns):
Area Code       170682 non-null int64
Area            170682 non-null object
Item Code       170682 non-null int64
Item            170682 non-null object
Element Code    170682 non-null int64
Element         170682 non-null object
Year Code       170682 non-null int64
Year            170682 non-null int64
Unit            170682 non-null object
Value           167239 non-null float64
Flag            118551 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 14.3+ MB


In [8]:
livestock.head()

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,2,Afghanistan,1107,Asses,5111,Stocks,1961,1961,Head,1300000.0,NaN
1,2,Afghanistan,1107,Asses,5111,Stocks,1962,1962,Head,851850.0,NaN
2,2,Afghanistan,1107,Asses,5111,Stocks,1963,1963,Head,1001112.0,NaN
3,2,Afghanistan,1107,Asses,5111,Stocks,1964,1964,Head,1150000.0,F
4,2,Afghanistan,1107,Asses,5111,Stocks,1965,1965,Head,1300000.0,NaN


In [9]:
livestock.tail()

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
170677,5817,Net Food Importing Developing Countries,1749,Sheep and Goats,5111,Stocks,2013,2013,Head,768797806.0,A
170678,5817,Net Food Importing Developing Countries,1749,Sheep and Goats,5111,Stocks,2014,2014,Head,787496839.0,A
170679,5817,Net Food Importing Developing Countries,1749,Sheep and Goats,5111,Stocks,2015,2015,Head,842294852.0,A
170680,5817,Net Food Importing Developing Countries,1749,Sheep and Goats,5111,Stocks,2016,2016,Head,871769775.0,A
170681,5817,Net Food Importing Developing Countries,1749,Sheep and Goats,5111,Stocks,2017,2017,Head,895402078.0,A


What are the distinct livestock items?

In [10]:
livestock['Item'].unique()

array(['Asses', 'Camels', 'Cattle', 'Chickens', 'Goats', 'Horses',
       'Mules', 'Sheep', 'Cattle and Buffaloes', 'Poultry Birds',
       'Sheep and Goats', 'Beehives', 'Buffaloes', 'Ducks',
       'Geese and guinea fowls', 'Pigs', 'Turkeys', 'Rabbits and hares',
       'Camelids, other', 'Rodents, other', 'Animals live nes',
       'Pigeons, other birds'], dtype=object)

What are the distinct years of the dataset?

In [11]:
livestock['Year'].unique()

array([1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017])

And what are the distinct units in the dataset?

In [12]:
livestock['Unit'].unique()

array(['Head', '1000 Head', 'No'], dtype=object)

Only beehives have a 'No' unit. Since we are not interested in these we'll ignore for now.

In [13]:
livestock.loc[livestock['Unit'] == 'No']['Item'].unique()

array(['Beehives'], dtype=object)

How much of the most common livestock resides in Belgium in 2017?

In [14]:
livestock.loc[(livestock['Item'].isin(['Cattle', 'Chickens', 'Pigs', 'Sheep'])) & (livestock['Year'] == 2017) & (livestock['Area'] == 'Belgium')]

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
10014,255,Belgium,866,Cattle,5111,Stocks,2017,2017,Head,2385988.0,NaN
10032,255,Belgium,1057,Chickens,5112,Stocks,2017,2017,1000 Head,26905.0,NaN
10122,255,Belgium,1034,Pigs,5111,Stocks,2017,2017,Head,6108077.0,NaN
10158,255,Belgium,976,Sheep,5111,Stocks,2017,2017,Head,85390.0,Im


We see that we have mixed units: 'Head' and '1000 Head'. Lets normalize...

In [15]:
multiplier_values = livestock['Unit'].transform(lambda x: 1000.0 if x == '1000 Head' else 1.0)

In [16]:
fixed_unit = livestock['Unit'].transform(lambda x: 'Head' if x == '1000 Head' else x)

In [17]:
normalized_values = livestock['Value'].mul(multiplier_values)

In [18]:
updated_values = pd.DataFrame({'Value': normalized_values, 'Unit': fixed_unit })

In [19]:
livestock.update(updated_values)

Lets see if the Belgian chickens are normalized now...

In [20]:
livestock[(livestock['Item'].isin(['Cattle', 'Chickens', 'Pigs', 'Sheep'])) & (livestock['Year'] == 2017) & (livestock['Area'] == 'Belgium')]

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
10014,255,Belgium,866,Cattle,5111,Stocks,2017,2017,Head,2385988.0,NaN
10032,255,Belgium,1057,Chickens,5112,Stocks,2017,2017,Head,26905000.0,NaN
10122,255,Belgium,1034,Pigs,5111,Stocks,2017,2017,Head,6108077.0,NaN
10158,255,Belgium,976,Sheep,5111,Stocks,2017,2017,Head,85390.0,Im


In [21]:
livestock[(livestock['Item'].isin(['Cattle', 'Chickens', 'Pigs', 'Sheep'])) & (livestock['Area'] == 'China')]

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
24773,351,China,866,Cattle,5111,Stocks,1961,1961,Head,49514057.0,A
24774,351,China,866,Cattle,5111,Stocks,1962,1962,Head,46875560.0,A
24775,351,China,866,Cattle,5111,Stocks,1963,1963,Head,47475507.0,A
24776,351,China,866,Cattle,5111,Stocks,1964,1964,Head,49056062.0,A
24777,351,China,866,Cattle,5111,Stocks,1965,1965,Head,52533452.0,A
...,...,...,...,...,...,...,...,...,...,...,...
25395,351,China,976,Sheep,5111,Stocks,2013,2013,Head,143680040.0,A
25396,351,China,976,Sheep,5111,Stocks,2014,2014,Head,150017440.0,A
25397,351,China,976,Sheep,5111,Stocks,2015,2015,Head,158490235.0,A
25398,351,China,976,Sheep,5111,Stocks,2016,2016,Head,162062714.0,A


In [22]:
livestock[(livestock['Item'].isin(['Cattle', 'Chickens', 'Pigs', 'Sheep']))]

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
114,2,Afghanistan,866,Cattle,5111,Stocks,1961,1961,Head,2900000.0,NaN
115,2,Afghanistan,866,Cattle,5111,Stocks,1962,1962,Head,3200000.0,F
116,2,Afghanistan,866,Cattle,5111,Stocks,1963,1963,Head,3300000.0,F
117,2,Afghanistan,866,Cattle,5111,Stocks,1964,1964,Head,3350000.0,F
118,2,Afghanistan,866,Cattle,5111,Stocks,1965,1965,Head,3400000.0,F
...,...,...,...,...,...,...,...,...,...,...,...
170449,5817,Net Food Importing Developing Countries,976,Sheep,5111,Stocks,2013,2013,Head,322567008.0,A
170450,5817,Net Food Importing Developing Countries,976,Sheep,5111,Stocks,2014,2014,Head,327792442.0,A
170451,5817,Net Food Importing Developing Countries,976,Sheep,5111,Stocks,2015,2015,Head,352050951.0,A
170452,5817,Net Food Importing Developing Countries,976,Sheep,5111,Stocks,2016,2016,Head,365697728.0,A


In [23]:
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11072 entries, 0 to 11071
Data columns (total 64 columns):
Series Name      11069 non-null object
Series Code      11067 non-null object
Country Name     11067 non-null object
Country Code     11067 non-null object
1960 [YR1960]    11067 non-null object
1961 [YR1961]    11067 non-null object
1962 [YR1962]    11067 non-null object
1963 [YR1963]    11067 non-null object
1964 [YR1964]    11067 non-null object
1965 [YR1965]    11067 non-null object
1966 [YR1966]    11067 non-null object
1967 [YR1967]    11067 non-null object
1968 [YR1968]    11067 non-null object
1969 [YR1969]    11067 non-null object
1970 [YR1970]    11067 non-null object
1971 [YR1971]    11067 non-null object
1972 [YR1972]    11067 non-null object
1973 [YR1973]    11067 non-null object
1974 [YR1974]    11067 non-null object
1975 [YR1975]    11067 non-null object
1976 [YR1976]    11067 non-null object
1977 [YR1977]    11067 non-null object
1978 [YR1978]    11067 non-null o

In [24]:
population['Series Name'].unique()

array(['Population, total', 'Population growth (annual %)',
       'Surface area (sq. km)',
       'Poverty headcount ratio at national poverty lines (% of population)',
       'GNI, Atlas method (current US$)',
       'GNI per capita, Atlas method (current US$)',
       'GNI, PPP (current international $)',
       'GNI per capita, PPP (current international $)',
       'Income share held by lowest 20%',
       'Life expectancy at birth, total (years)',
       'Fertility rate, total (births per woman)',
       'Adolescent fertility rate (births per 1,000 women ages 15-19)',
       'Contraceptive prevalence, any methods (% of women ages 15-49)',
       'Births attended by skilled health staff (% of total)',
       'Mortality rate, under-5 (per 1,000 live births)',
       'Prevalence of underweight, weight for age (% of children under 5)',
       'Immunization, measles (% of children ages 12-23 months)',
       'Primary completion rate, total (% of relevant age group)',
       'School en

In [25]:
population[(population['Series Name'] == 'Population, total') & (population['Country Name'] == 'Belgium')]

,Series Name,Series Code,Country Name,Country Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],...,2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019]
18,"Population, total",SP.POP.TOTL,Belgium,BEL,9153489,9183948,9220578,9289770,9378113,9463667,...,10895586,11038264,11106932,11159407,11209057,11274196,11331422,11375158,11422068,..


First we drop the columns we do not need from population

In [26]:
pop = population[(population['Series Name'] == 'Population, total')].drop(['Series Name', 'Series Code', 'Country Code'], axis=1)

In [27]:
pop.tail()

,Country Name,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],1966 [YR1966],1967 [YR1967],1968 [YR1968],...,2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019]
212,Virgin Islands (U.S.),32500,34300,35000,39800,40800,43500,46200,49100,55700,...,108358,108292,108191,108044,107884,107710,107510,107268,106977,..
213,West Bank and Gaza,..,..,..,..,..,..,..,..,..,...,3786161,3882986,3979998,4076708,4173398,4270092,4367088,4454805,4569087,..
214,Yemen,5315355,5393036,5473671,5556766,5641597,5727751,5816247,5907874,6001852,...,23154855,23807588,24473178,25147109,25823485,26497889,27168210,27834821,28498687,..
215,Zambia,3070776,3164329,3260650,3360104,3463213,3570464,3681955,3797873,3918872,...,13605984,14023193,14465121,14926504,15399753,15879361,16363507,16853688,17351822,..
216,Zimbabwe,3776681,3905034,4039201,4178726,4322861,4471177,4623351,4779827,4941906,...,12697723,12894316,13115131,13350356,13586681,13814629,14030390,14236745,14439018,..


Get rid of those weird '..' empty values

In [28]:
pop.replace('..', np.NaN, inplace=True)

Let's first unpivot all the year columns from the population dataset

In [29]:
unpivot_pop = pd.melt(pop, id_vars=['Country Name'], var_name="Year", value_name="Count")

In [30]:
unpivot_pop['Year'] = unpivot_pop['Year'].map(lambda x: int(re.sub(r" \[.*\]", "", x)))

In [31]:
unpivot_pop[unpivot_pop['Country Name'] == 'West Bank and Gaza'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 213 to 13016
Data columns (total 3 columns):
Country Name    60 non-null object
Year            60 non-null int64
Count           29 non-null object
dtypes: int64(1), object(2)
memory usage: 1.9+ KB


In [32]:
meat_balance_sheet.head()

,Domain Code,Domain,Country Code,Country,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FBS,Food Balance Sheets,2,Afghanistan,5511,Production,2731,Bovine Meat,1961,1961,1000 tonnes,43,S,Standardized data
1,FBS,Food Balance Sheets,2,Afghanistan,5511,Production,2731,Bovine Meat,1962,1962,1000 tonnes,46,S,Standardized data
2,FBS,Food Balance Sheets,2,Afghanistan,5511,Production,2731,Bovine Meat,1963,1963,1000 tonnes,47,S,Standardized data
3,FBS,Food Balance Sheets,2,Afghanistan,5511,Production,2731,Bovine Meat,1964,1964,1000 tonnes,48,S,Standardized data
4,FBS,Food Balance Sheets,2,Afghanistan,5511,Production,2731,Bovine Meat,1965,1965,1000 tonnes,49,S,Standardized data


First rename the 'Country' column name to 'Country Name' to align with the population DataFrame

In [33]:
meat_balance_sheet.rename(columns={'Country': 'Country Name'}, inplace=True)

In [34]:
meat_balance_sheet['Element'].unique()

array(['Production', 'Import Quantity', 'Domestic supply quantity',
       'Export Quantity'], dtype=object)

In [35]:
meat_balance_sheet['Item'].unique()

array(['Bovine Meat', 'Mutton & Goat Meat', 'Poultry Meat', 'Pigmeat'],
      dtype=object)

In [36]:
meat_balance_sheet.loc[(meat_balance_sheet['Item'] == 'Poultry Meat') & (meat_balance_sheet['Country Name'] == 'China') & (meat_balance_sheet['Element'] == 'Domestic supply quantity')].head()

,Domain Code,Domain,Country Code,Country Name,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
26168,FBS,Food Balance Sheets,41,China,5301,Domestic supply quantity,2734,Poultry Meat,1961,1961,1000 tonnes,619,S,Standardized data
26169,FBS,Food Balance Sheets,41,China,5301,Domestic supply quantity,2734,Poultry Meat,1962,1962,1000 tonnes,646,S,Standardized data
26170,FBS,Food Balance Sheets,41,China,5301,Domestic supply quantity,2734,Poultry Meat,1963,1963,1000 tonnes,670,S,Standardized data
26171,FBS,Food Balance Sheets,41,China,5301,Domestic supply quantity,2734,Poultry Meat,1964,1964,1000 tonnes,690,S,Standardized data
26172,FBS,Food Balance Sheets,41,China,5301,Domestic supply quantity,2734,Poultry Meat,1965,1965,1000 tonnes,719,S,Standardized data


Which countries do not match between the population and meat balance DataFrames?

In [37]:
set(meat_balance_sheet['Country Name']).difference(set(unpivot_pop['Country Name']))

{'China, Taiwan Province of',
 'Congo',
 'Czechia',
 'Czechoslovakia',
 "CÃ´te d'Ivoire",
 'Ethiopia',
 "Lao People's Democratic Republic",
 'Netherlands Antilles (former)',
 'Saint Kitts and Nevis',
 'Saint Lucia',
 'Saint Vincent and the Grenadines',
 'Serbia and Montenegro',
 'Sudan (former)',
 'USSR',
 'Yugoslav SFR'}

In [38]:
set(unpivot_pop['Country Name']).difference(set(meat_balance_sheet['Country Name']))

{'American Samoa',
 'Andorra',
 'Aruba',
 'Bahrain',
 'Bhutan',
 'British Virgin Islands',
 'Burundi',
 'Cayman Islands',
 'Channel Islands',
 'Comoros',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 'Curacao',
 'Czech Republic',
 "CÃ\x83Â´te d'Ivoire",
 'Equatorial Guinea',
 'Eritrea',
 'Faroe Islands',
 'Gibraltar',
 'Greenland',
 'Guam',
 'Isle of Man',
 'Kosovo',
 'Lao PDR',
 'Libya',
 'Liechtenstein',
 'Marshall Islands',
 'Micronesia, Fed. Sts.',
 'Monaco',
 'Nauru',
 'Northern Mariana Islands',
 'Palau',
 'Papua New Guinea',
 'Puerto Rico',
 'Qatar',
 'San Marino',
 'Seychelles',
 'Singapore',
 'Sint Maarten (Dutch part)',
 'Somalia',
 'South Sudan',
 'St. Kitts and Nevis',
 'St. Lucia',
 'St. Martin (French part)',
 'St. Vincent and the Grenadines',
 'Syrian Arab Republic',
 'Tonga',
 'Turks and Caicos Islands',
 'Tuvalu',
 'Virgin Islands (U.S.)',
 'West Bank and Gaza'}

In [39]:
meat_balance_sheet['Country Name'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bangladesh', 'Barbados',
       'Belarus', 'Belgium', 'Luxembourg', 'Belize', 'Benin', 'Bermuda',
       'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria',
       'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile',
       'China, Hong Kong SAR', 'China, Macao SAR', 'China',
       'China, Taiwan Province of', 'Colombia', 'Congo', 'Costa Rica',
       "CÃ´te d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia',
       'Czechoslovakia', "Democratic People's Republic of Korea",
       'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Estonia', 'Ethiopia', 'Ethiopia PDR',
       'Fiji', 'Finland', 'France', 'French Polynesia', 'Gabon', 'Gambia',
 

Now we import the countries population sizes dataset and normalize the common livestock per person in a country.

When all data is normalized and livestock is tempered by export figures we can view the evolution of type of livestock consumption per capita head in a sorted stream graph.

Joining the 2 datasets to get populations agains domestic supply  quantity: join_pop_pd.merge(df1,df2,on='key')

In [40]:
joined_meat_pop = pd.merge(unpivot_pop, meat_balance_sheet, on=['Country Name', 'Year'])

In [54]:
joined_meat_pop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120299 entries, 0 to 120334
Data columns (total 16 columns):
Country Name        120299 non-null object
Year                120299 non-null int64
Count               120299 non-null object
Domain Code         120299 non-null object
Domain              120299 non-null object
Country Code        120299 non-null int64
Element Code        120299 non-null int64
Element             120299 non-null object
Item Code           120299 non-null int64
Item                120299 non-null object
Year Code           120299 non-null int64
Unit                120299 non-null object
Value               120299 non-null int64
Flag                120299 non-null object
Flag Description    120299 non-null object
Capita              120299 non-null int64
dtypes: int64(7), object(9)
memory usage: 15.6+ MB


In [53]:
joined_meat_pop.dropna(inplace=True)

In [59]:
joined_meat_pop['Capita'] = 10**6 * joined_meat_pop['Value'] / joined_meat_pop['Count'].map(lambda x: int(x))

In [57]:
joined_meat_pop[joined_meat_pop['Country Name'] == 'Belgium']

,Country Name,Year,Count,Domain Code,Domain,Country Code,Element Code,Element,Item Code,Item,Year Code,Unit,Value,Flag,Flag Description,Capita
85882,Belgium,2000,10251250,FBS,Food Balance Sheets,255,5511,Production,2731,Bovine Meat,2000,1000 tonnes,275,S,Standardized data,0.026826
85883,Belgium,2000,10251250,FBS,Food Balance Sheets,255,5611,Import Quantity,2731,Bovine Meat,2000,1000 tonnes,49,S,Standardized data,0.004780
85884,Belgium,2000,10251250,FBS,Food Balance Sheets,255,5911,Export Quantity,2731,Bovine Meat,2000,1000 tonnes,125,S,Standardized data,0.012194
85885,Belgium,2000,10251250,FBS,Food Balance Sheets,255,5301,Domestic supply quantity,2731,Bovine Meat,2000,1000 tonnes,200,S,Standardized data,0.019510
85886,Belgium,2000,10251250,FBS,Food Balance Sheets,255,5511,Production,2732,Mutton & Goat Meat,2000,1000 tonnes,5,S,Standardized data,0.000488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118053,Belgium,2013,11159407,FBS,Food Balance Sheets,255,5301,Domestic supply quantity,2733,Pigmeat,2013,1000 tonnes,429,S,Standardized data,0.038443
118054,Belgium,2013,11159407,FBS,Food Balance Sheets,255,5511,Production,2734,Poultry Meat,2013,1000 tonnes,389,S,Standardized data,0.034858
118055,Belgium,2013,11159407,FBS,Food Balance Sheets,255,5611,Import Quantity,2734,Poultry Meat,2013,1000 tonnes,252,S,Standardized data,0.022582
118056,Belgium,2013,11159407,FBS,Food Balance Sheets,255,5911,Export Quantity,2734,Poultry Meat,2013,1000 tonnes,498,S,Standardized data,0.044626


In [52]:
joined_meat_pop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120335 entries, 0 to 120334
Data columns (total 15 columns):
Country Name        120335 non-null object
Year                120335 non-null int64
Count               120299 non-null object
Domain Code         120335 non-null object
Domain              120335 non-null object
Country Code        120335 non-null int64
Element Code        120335 non-null int64
Element             120335 non-null object
Item Code           120335 non-null int64
Item                120335 non-null object
Year Code           120335 non-null int64
Unit                120335 non-null object
Value               120335 non-null int64
Flag                120335 non-null object
Flag Description    120335 non-null object
dtypes: int64(6), object(9)
memory usage: 14.7+ MB


In [65]:
joined_meat_pop[(joined_meat_pop['Element'] == 'Domestic supply quantity') & (joined_meat_pop['Year'] == 2010) & (joined_meat_pop['Country Name'] == 'Netherlands')]

,Country Name,Year,Count,Domain Code,Domain,Country Code,Element Code,Element,Item Code,Item,Year Code,Unit,Value,Flag,Flag Description,Capita
112004,Netherlands,2010,16615394,FBS,Food Balance Sheets,150,5301,Domestic supply quantity,2731,Bovine Meat,2010,1000 tonnes,315,S,Standardized data,18.958323
112008,Netherlands,2010,16615394,FBS,Food Balance Sheets,150,5301,Domestic supply quantity,2732,Mutton & Goat Meat,2010,1000 tonnes,15,S,Standardized data,0.902777
112012,Netherlands,2010,16615394,FBS,Food Balance Sheets,150,5301,Domestic supply quantity,2733,Pigmeat,2010,1000 tonnes,563,S,Standardized data,33.884240
112016,Netherlands,2010,16615394,FBS,Food Balance Sheets,150,5301,Domestic supply quantity,2734,Poultry Meat,2010,1000 tonnes,311,S,Standardized data,18.717582


In [125]:
result = joined_meat_pop[['Country Name', 'Year', 'Element', 'Item', 'Capita']]

In [109]:
pigmeat = result[(result['Country Name'] == 'Belgium') & (result['Element'] == 'Domestic supply quantity')]

In [110]:
usa = result[(result['Country Name'] == 'United States of America') & (result['Element'] == 'Domestic supply quantity')]

In [117]:
cambodia = result[(result['Country Name'] == 'Cambodia') & (result['Element'] == 'Domestic supply quantity')]

In [122]:
fig, ax = plt.subplots()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

In [124]:
fig.suptitle('Pigmeat consumption per capita', fontsize=14)

Text(0.5,0.98,'Pigmeat consumption per capita')

In [119]:
cambodia[cambodia['Item'] == 'Pigmeat'].plot(ax=ax, x='Year', y='Capita', label='Cambodia')

In [120]:
belgium[belgium['Item'] == 'Pigmeat'].plot(ax=ax, x='Year', y='Capita', label='Belgium')

In [121]:
usa[usa['Item'] == 'Pigmeat'].plot(ax=ax, x='Year', y='Capita', label='USA')